# **Test RF with Multivariate Predictive Clustering Trees of Spyct package**

https://github.com/knowledge-technologies/spyct


```
@misc{stepinik2020oblique,
    title={Oblique Predictive Clustering Trees},
    author={Tomaž Stepišnik and Dragi Kocev},
    year={2020},
    eprint={2007.13617},
    archivePrefix={arXiv},
    primaryClass={cs.LG}
}
```



In [1]:
pip install git+https://gitlab.com/TStepi/spyct.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://gitlab.com/TStepi/spyct.git to /tmp/pip-req-build-m52dv0by
  Running command git clone --filter=blob:none --quiet https://gitlab.com/TStepi/spyct.git /tmp/pip-req-build-m52dv0by
  Resolved https://gitlab.com/TStepi/spyct.git to commit 0bc9808a41f49ebb7adbd26c63965b7bd108903e
  Preparing metadata (setup.py) ... done
  Created wheel for spyct-tstepi: filename=spyct_tstepi-1.0-cp38-cp38-linux_x86_64.whl size=2164796 sha256=84ad54233aa9e29ed36d259047b2f325a45c771c8f6dc1106850b4a966bdc53d
  Stored in directory: /tmp/pip-ephem-wheel-cache-0h009o5u/wheels/85/52/c0/fff548d9a836446adbbf1d985ac420f941d46bac779b8a786f
Successfully built spyct-tstepi


In [11]:
FOLDER_ROOT = '/content/drive/MyDrive'
import sys
sys.path.append(FOLDER_ROOT)

import pandas as pd

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix

from midas.v4.ensemble import RFClassifier

import spyct

In [19]:
df_stream = pd.read_csv('https://github.com/scikit-multiflow/streaming-datasets/raw/master/agr_a.csv',
                        engine='c', low_memory=True, memory_map=True)

X = df_stream[df_stream.columns[:-1]]
y = df_stream['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.70, random_state=100)

ValueError: ignored

Testa o código para ver se funciona

In [ ]:
# Fit the model and make predictions
model = spyct.Model()
model.fit(X_train.to_numpy(copy=True), y_train.to_numpy(copy=True).reshape(-1, 1))
y_pred = model.predict(X_test.to_numpy(copy=True))

# decode the encoded predictions
y_pred = np.argmax(y_pred, axis=1)

# Show the confusion matrix
print(confusion_matrix(y_test, y_pred))

[[369561      0]
 [330439      0]]


In [ ]:
accuracy_score(y_test, y_pred)

0.5279442857142858

In [8]:
from sklearn.datasets import load_iris

# Load the iris dataset from scikit learn.
dataset = load_iris()
X = dataset.data
y = dataset.target.reshape(-1, 1)


# Split the data into train and test subsets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# For multi-class datasets, target needs to be one-hot encoded.
encoder = OneHotEncoder()
y_train = encoder.fit_transform(y_train).toarray()

# Fit the model and make predictions
model = spyct.Model()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# decode the encoded predictions
y_pred = np.argmax(y_pred, axis=1)

# Show the confusion matrix
print(confusion_matrix(y_test, y_pred))

[[ 9  0  0]
 [ 0 13  1]
 [ 0  0  7]]


In [9]:
np.unique(y_train)

array([0., 1.])

In [10]:
y_pred

array([1, 1, 0, 0, 2, 2, 0, 0, 1, 1, 2, 0, 1, 2, 1, 2, 0, 1, 2, 2, 1, 1,
       2, 0, 1, 1, 0, 0, 1, 1])

Testa com o RFClassifier

In [16]:
cfg_base_estimator = {'splitter':'grad', # 'grad' or 'svm'
                      'num_trees': 100,
                      'max_features': 1.0, # int, float, "sqrt", "log"
                      'bootstrapping':None, # True or False
                      'max_depth': np.inf, # or int
                      'min_examples_to_split': 2,
                      'min_impurity_decrease': 0, # or float
                      'n_jobs':1, # 1
                      'standardize_descriptive': True,
                      'standardize_clustering': True,
                      'max_iter':100,
                      'lr': 0.01, # or float
                      'C': 0, # or float
                      'balance_classes': True,
                      'tol': 0, # or float
                      'eps':1e-8, # or float
                      #adam_beta1: 0.9, # or float if solitter='grad'
                      #adam_beta2: 0.999, # or float if solitter='grad'
                      'random_state':100, # RandomState instance, int, (default=None)
                      }

rf = RFClassifier(base_estimator=spyct.Model,
                  n_estimators=100,
                  params_estimators=cfg_base_estimator,
                  enable_logger=True)

rf._setup_logger(log_file='/content/drive/MyDrive/Testes e Experimentos/Testes/rf.spyct.debuging.log')

rf = rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print('Acc with oblique tree', accuracy_score(y_test, y_pred))

AttributeError: ignored